In [ ]:
import bpy
import numpy as np
import mathutils
import math

# ============================================
# CONFIGURATION
# ============================================
NPZ_PATH = "C:/projects/FYP/experiments/generated_output.npz" 
ARMATURE_NAME = "Armature"
FRAME_START = 1

# --- SMOOTHING (Fixes Jitter) ---
# 0.0 = No smoothing (raw data), 0.9 = Very smooth but potentially "laggy"
# Recommended: 0.4 to 0.7
SMOOTHING_FACTOR = 0.9

# --- INDIVIDUAL FINGER PROFILES ---
FINGER_CONFIG = {
    'thumb': {
        'curl_boost': 0.1, 'across_boost': 1.2, 'twist': 0.6,
        'limits': {'BASE': 100, 'MID': 90, 'TIP': 70, 'BACK': -20}
    },
    'index': {
        'curl_boost': 2.0, 'spread': 0.7, 'twist': 0.05,
        'limits': {'BASE': 90, 'MID': 100, 'TIP': 80, 'BACK': -15}
    },
    'middle': {
        'curl_boost': 2.8, 'spread': 0.6, 'twist': 0.05, 
        'limits': {'BASE': 110, 'MID': 120, 'TIP': 100, 'BACK': -20} 
    },
    'ring': {
        'curl_boost': 2.4, 'spread': 0.6, 'twist': 0.05,
        'limits': {'BASE': 95, 'MID': 110, 'TIP': 90, 'BACK': -15}
    },
    'pinky': {
        'curl_boost': 2.4, 'spread': 0.9, 'twist': 0.05,
        'limits': {'BASE': 100, 'MID': 110, 'TIP': 100, 'BACK': -15}
    }
}

BEND_DIRECTION = 1.0  
ANIMATE_LEFT_SIDE = False  
IDLE_STRENGTH = 0.03       
IDLE_SPEED = 0.1           

FINGER_MAPPING = {
    'Thumb1': (0, 1),   'Thumb2': (1, 2),   'Thumb3': (2, 3),   'Thumb4': (3, 4),
    'Index1': (5, 6),   'Index2': (6, 7),   'Index3': (7, 8),   'Index4': (8, 8),
    'Middle1': (9, 10), 'Middle2': (10, 11), 'Middle3': (11, 12), 'Middle4': (12, 12),
    'Ring1': (13, 14),  'Ring2': (14, 15),  'Ring3': (15, 16),  'Ring4': (16, 16),
    'Pinky1': (17, 18), 'Pinky2': (18, 19), 'Pinky3': (19, 20), 'Pinky4': (20, 20),
}

# Dictionary to store previous rotations for smoothing
last_rotations = {}

# ============================================
# SOLVERS
# ============================================

def np_to_vec(coords):
    return mathutils.Vector((coords[0], coords[2], -coords[1]))

def find_pose_bone(armature, name_variants):
    for name in name_variants:
        pb = armature.pose.bones.get(name)
        if pb: return pb
    return None

def apply_smoothing(bone_name, target_quat):
    """Blends the current target rotation with the previous frame's rotation."""
    if bone_name in last_rotations:
        prev_quat = last_rotations[bone_name]
        # Slerp: Spherical Linear Interpolation
        # factor 1.0 = target_quat, factor 0.0 = prev_quat
        smoothed_quat = prev_quat.slerp(target_quat, 1.0 - SMOOTHING_FACTOR)
        last_rotations[bone_name] = smoothed_quat
        return smoothed_quat
    else:
        last_rotations[bone_name] = target_quat
        return target_quat

def solve_generic_rotation(bone, vec_start, vec_end):
    target_dir = (vec_end - vec_start).normalized()
    if target_dir.length < 0.0001: return mathutils.Quaternion((1, 0, 0, 0))
    parent_mat = bone.parent.matrix.to_3x3() if bone.parent else mathutils.Matrix.Identity(3)
    local_target = parent_mat.inverted() @ target_dir
    bone_rest = bone.bone.matrix_local.to_3x3()
    rest_diff = bone.parent.bone.matrix_local.to_3x3().inverted() @ bone_rest if bone.parent else bone_rest
    local_rest = (rest_diff @ mathutils.Vector((0, 1, 0))).normalized()
    return local_rest.rotation_difference(local_target)

def solve_safe_joint(bone, vec_start, vec_end):
    target_dir = (vec_end - vec_start).normalized()
    if target_dir.length < 0.0001: return mathutils.Quaternion((1, 0, 0, 0))
    parent_mat = bone.parent.matrix.to_3x3() if bone.parent else mathutils.Matrix.Identity(3)
    local_target = parent_mat.inverted() @ target_dir
    bone_rest = bone.bone.matrix_local.to_3x3()
    rest_diff = bone.parent.bone.matrix_local.to_3x3().inverted() @ bone_rest if bone.parent else bone_rest
    local_rest = (rest_diff @ mathutils.Vector((0, 1, 0))).normalized()
    quat = local_rest.rotation_difference(local_target)
    euler = quat.to_euler('YZX')

    bone_name_lower = bone.name.lower()
    f_type = next((f for f in FINGER_CONFIG.keys() if f in bone_name_lower), 'index')
    cfg = FINGER_CONFIG[f_type]

    if f_type != 'thumb':
        euler.x *= (BEND_DIRECTION * cfg['curl_boost'])
        if "1" in bone.name:   max_x = math.radians(cfg['limits']['BASE'])
        elif "2" in bone.name: max_x = math.radians(cfg['limits']['MID'])
        else:                  max_x = math.radians(cfg['limits']['TIP'])
        euler.x = max(min(euler.x, max_x), math.radians(cfg['limits']['BACK']))
        if "1" in bone.name:
            euler.y *= cfg['twist']; euler.z *= cfg['spread']
        else:
            euler.y *= 0.01; euler.z *= 0.01 
    else:
        euler.x *= (BEND_DIRECTION * cfg['curl_boost'])
        euler.z *= cfg['across_boost']; euler.y *= cfg['twist']
        euler.x = max(min(euler.x, math.radians(cfg['limits']['BASE'])), math.radians(cfg['limits']['BACK']))

    return euler.to_quaternion()

def solve_hand_rotation(p_wrist, p_index, p_pinky, bone, is_left):
    v_index = (p_index - p_wrist).normalized()
    v_pinky = (p_pinky - p_wrist).normalized()
    palm_normal = v_pinky.cross(v_index).normalized() if is_left else v_index.cross(v_pinky).normalized()
    hand_dir = (v_index + v_pinky).normalized() 
    hand_side = hand_dir.cross(palm_normal).normalized()
    palm_normal = hand_side.cross(hand_dir).normalized()
    target_matrix = mathutils.Matrix((hand_side, hand_dir, palm_normal)).transposed()
    parent_mat = bone.parent.matrix.to_3x3() if bone.parent else mathutils.Matrix.Identity(3)
    target_local = parent_mat.inverted() @ target_matrix
    bone_rest = bone.bone.matrix_local.to_3x3()
    rest_local = bone.parent.bone.matrix_local.to_3x3().inverted() @ bone_rest if bone.parent else bone_rest
    return (rest_local.inverted() @ target_local).to_quaternion()

# ============================================
# MAIN LOOP
# ============================================

def run_refined_animation():
    global last_rotations
    last_rotations = {} # Clear for new run
    
    try:
        data = np.load(NPZ_PATH)
        pose_data, lh_data, rh_data = data['pose'], data['lh'], data['rh']
    except Exception as e:
        print(f"Error loading NPZ: {e}"); return

    num_frames = len(pose_data)
    bpy.context.scene.frame_start = FRAME_START
    bpy.context.scene.frame_end = FRAME_START + num_frames - 1

    armature = bpy.data.objects.get(ARMATURE_NAME)
    if not armature: return
    
    bpy.context.view_layer.objects.active = armature
    bpy.ops.object.mode_set(mode='POSE')
    if armature.animation_data: armature.animation_data.action = None

    for f in range(num_frames):
        current_frame = FRAME_START + f
        idle_w = math.sin(f * IDLE_SPEED) * IDLE_STRENGTH
        idle_h = math.cos(f * IDLE_SPEED * 0.7) * IDLE_STRENGTH
        
        # 1. ARMS
        for b_name, s, e in [('LeftArm', 11, 13), ('LeftForeArm', 13, 15), ('RightArm', 12, 14), ('RightForeArm', 14, 16)]:
            pb = armature.pose.bones.get(b_name)
            if pb:
                if not ANIMATE_LEFT_SIDE and b_name.startswith('Left'):
                    v_start, v_end = mathutils.Vector((0,0,0)), mathutils.Vector((0.15+idle_w, idle_h, -1.0+idle_w))
                    target_rot = solve_generic_rotation(pb, v_start, v_end)
                else:
                    target_rot = solve_generic_rotation(pb, np_to_vec(pose_data[f][s]), np_to_vec(pose_data[f][e]))
                
                # Apply Smoothing
                pb.rotation_quaternion = apply_smoothing(b_name, target_rot)
                pb.keyframe_insert("rotation_quaternion", frame=current_frame)

        # 2. HANDS & FINGERS
        for prefix, h_pts, is_left in [('Left', lh_data[f], True), ('Right', rh_data[f], False)]:
            if is_left and not ANIMATE_LEFT_SIDE: continue 
            if np.all(h_pts == 0): continue
            
            # Wrist
            wrist_pb = armature.pose.bones.get(f'{prefix}Hand')
            if wrist_pb:
                raw_wrist = solve_hand_rotation(np_to_vec(h_pts[0]), np_to_vec(h_pts[5]), np_to_vec(h_pts[17]), wrist_pb, is_left)
                wrist_pb.rotation_quaternion = apply_smoothing(f"{prefix}Wrist", raw_wrist)
                wrist_pb.keyframe_insert("rotation_quaternion", frame=current_frame)
            
            bpy.context.view_layer.update()

            # Fingers
            for fname, (s, e) in FINGER_MAPPING.items():
                variants = [f"{prefix}Hand{fname}", f"{prefix}Hand.{fname}", f"{prefix}Hand_{fname}", fname]
                pb = find_pose_bone(armature, variants)
                if pb:
                    v_start, v_end = np_to_vec(h_pts[s]), np_to_vec(h_pts[e])
                    if (v_end - v_start).length < 0.001 and s > 0:
                        v_end = v_start + (v_start - np_to_vec(h_pts[s-1])).normalized() * 0.02
                    
                    raw_finger = solve_safe_joint(pb, v_start, v_end)
                    # Unique key for each finger bone in last_rotations
                    pb.rotation_quaternion = apply_smoothing(f"{prefix}{fname}", raw_finger)
                    pb.keyframe_insert("rotation_quaternion", frame=current_frame)

    bpy.ops.object.mode_set(mode='OBJECT')
    print(f"✅ Success: Animation smoothed (Factor: {SMOOTHING_FACTOR}). Jitter should be reduced.")

if __name__ == "__main__":
    run_refined_animation()
